# Exploring prediction markets
> I want to explore some ideas about making bets

- toc: true 
- badges: true
- comments: true
- categories: [prediction markets, betting]

# Introduction to ideas

Alice and Bob are rational agents with their own current beliefs about whether Mr. T. will win the election. Mr. T. will either win or not, and so they view it as a Bernoulli variable $X$. Truly $X \sim \text{Ber}(\theta_{true})$.

Alice and Bob both have point estimates about the probability. They are $\theta_{a}$ and $\theta_{b}$ respectively, and therefore their beliefs are represented by their belief distributions $p_{a}(x) = \text{Ber}(x; \theta_{a})$ and $p_{b}(x) = \text{Ber}(x; \theta_{b})$.

How can Alice and Bob set up a bet they are both satisfied with?

# Pot splitting

The first idea Alice and Bob have is the following: They both put 50 cent in a pot, so now there is 1 dollar in the pot. When the election is over Alice and Bob will be honest with each about who truly won the election, so after the outcome of $X$ is revealed there will be no dispute.

How will they split the pot? Their first idea is the following. A gets fraction $f_a$ of the outcome,

$$
f_b(x) = \frac{p_a(x)}{p_a(x) + p_b(x)}
$$

Then *given* that A has belief $p_a$, A should expect to win
$$
\mathbb{E}_{x \sim p_a}[f_a(x)] = \theta_a \frac{\theta_a}{\theta_a + \theta_b} + (1-\theta_a) \frac{1 - \theta_a}{(1-\theta_a) + (1-\theta_b)}
$$

Likewise B expects $\mathbb{E}_{x \sim p_b}[f_b]$. Making a rule that $\theta_a$ and $\theta_b$ should be in the open interval $(0,1)$ - no absolute certainty - is sufficient condition for this expectation being defined, since then there is no division by zero. Also, only a Sith deals in absolutes.

If instead of just two players and two outcomes, there are $M$ players and $K$ outcomes (a categorical variable), then an equivalent rule for how much each player $m$ should get would be
$$
f_m(k) = \frac{p_m(k)}{\sum_{i=1}^{M} p_i(k)}
$$
and player $m$ would expect to win
$$\mathbb{E}_{k \sim p_m}[f_m(k)] = \sum_{k=1}p_m(k) f_m(k) = \sum_{k=1} p_m(k) \frac{p_m(k)}{\sum_{i=1}^{M} p_i(k)}$$

We could call $\mathbb{E}_{k \sim p_m}[f_m(k)]$ the "open self-expected prize" for player $m$ and denote it $Z_m$. It is "open" because it is a function of all the probabilities, so Alice would only be able to compute it if she knew the probabilities of all the others. It is "self-expected" because Alice computes the expectation with respect to her own beliefs about the outcome.

# Would Alice and Bob participate in such a bet?

If the $m$ players contribute an equal amount to the pot, then the fraction of the pot they will have at stake is $\frac{1}{M}$ dollars. Alice realized the same thing, that she expects to win $Z_a$. She is willing to participate in the bet if she does not expect to win less than what she has at stake. That means, a player $m$ is willing to participate if
$$
Z_m \geq \frac{1}{M}
$$

For finite number of agents $M$ and finite number of outcomes $K$, this inequality always holds. See [this MathOverflow post](https://mathoverflow.net/questions/416333/inequality-for-matrix-with-rows-summing-to-1). Thanks Federico Poloni and Iosif Pinelis!

# Is such a bet realistic?

Intuitively this tells us, that no matter what Bob thought, Alice would be able to compute $Z_a$ and come out in favor of participating in the bet. Likewise would Bob.

# If Bob said his belief, Alice could want to lie!

Let's say that Bob told Alice that his belief was $\theta_b = .3$. Assume that truly Alice has belief $\theta_a = .5$.
We can view all of this as a matrix $P$ showing the belief.

In [51]:
#collapse_hide
import torch
P = torch.tensor([
    [.7,.3],
    [.5,.5]
])
print('P:',P)

P: tensor([[0.7000, 0.3000],
        [0.5000, 0.5000]])


Then $Z_m$ is computed as follows

In [52]:
def Z(m,P):
    return P[m] @ (P[m] / P.sum(dim=0))

Alice and Bob have exactly the same $Z$. For more players than two, this will not be the case.

In [53]:
#collapse_hide
print('Z_a : ', Z(0,P).item())
print('Z_b : ', Z(1,P).item())

Z_a :  0.5208333134651184
Z_b :  0.5208333134651184


But we just said that Bob said his belief first, so now the question is - would Alice want to lie about her beliefs in order to win more? To think about this we have to distinguish between the belief-distribution and the commit-distribution. For now we assume that players never change their real beliefs about the outcome, but they might present a different distribution:

We write $q_a = \text{Ber}(\theta'_a)$ to denote the distribution with which Alice participates in the bet, so she will get

$$f'_a(x) = \frac{q_a(x)}{q_a(x) + q_b(x)}$$

With this change we can look at what a player $m$ expects to win $W_m = \mathbb{E}_{x \sim p_m}[f'_m(x)]$. The expectation is with respect to what the player truly belives about what will happen, but the public distribution $q_m(x)$ is what is used to compute the fraction. 

In [54]:
def W(m,P,Q):
    return P[m] @ (Q[m] / Q.sum(dim=0))

What if Alice chose to lie, and say that her belief is $.51$ instead of $.5$? Then her self-expectation $W_a > Z_a$, she would be better off by being dishonest.

In [59]:
#collapse_hide
import torch
Q = torch.tensor([
    [.7,.3],
    [.49,.51]
])
print('Q:', Q)

Q: tensor([[0.7000, 0.3000],
        [0.4900, 0.5100]])


In [60]:
#collapse_hide
print('Q_a : ', W(0,P,Q).item())
print('Q_b : ', W(1,P,Q).item())

Q_a :  0.5228757858276367
Q_b :  0.5206971168518066
